In this script, I try to use RL to solve the knapsack problem.
This should not be an exciting project but just a warm up (for me at least) to understand hands-on RL, RNN, embeddings and such.

The project starts with:
- a nn to move from 2 features per object to 8
- a LTSM with an internal state composed by 5 features
- its output is fed to a fully connected layer that from 5 feature expands to the number of objects
- a softmax decides which element to choose
- If you chose objects whose weight is more than 1, the reward is -100
- attention mechanism with glimpse

Still to implement:
- mask to not choose 'wrong' objects
- for now, it works only wwith batches of one. Cahnge this

As always, we start by importing useful libraries and defiening some boundary conditions

In [2]:
import matplotlib.pyplot as plt

import numpy as np 
import pandas as pd
import torch 
import random
import itertools    

%matplotlib inline

random.seed(1234)

Then we define the object class

In [3]:
class ObjectCass:
    
    Prize = None
    Weight = None
    
    def __init__(self, reward, weight):
        
        self.Prize = reward
        self.Weight = weight

Now we define the function to generate new instances.

In [4]:
def GenerateANewInstance(num_objs):
    
    # fake code to control the instance I fed it.
    # best solution = [1,0,1,0,1,0,1,0,1,0,1]
    
    Objects = []
    for i in range(num_objs):
        if i%2==1:
            prize = 0.1
            weight = 0.51
            obj = ObjectCass(prize,weight)
            Objects.append(obj)
        else:
            prize = 1
            weight = 0.19
            obj = ObjectCass(prize,weight)
            Objects.append(obj)
    Objects.append(ObjectCass(0,0)) # fake object representing that you don't pick up anything
    
    return Objects
    
    # real code    
    
#    Objects = []
#    for i in range(num_objs):
#        prize = round(random.uniform(prize_min, prize_max), 0)
#        weight = round(random.uniform(0.01, 1), 2)
#        obj = ObjectCass(prize,weight)
#        Objects.append(obj)
#    Objects.append(ObjectCass(0,0)) # fake object representing that you don't pick up anything
#                                   # i.e. you stop picking up stuff
#    
#    return Objects

Actor Net. input dimension [num_objects] output dimensions [num_objects]

In [15]:
class ActorNet(torch.nn.Module):
    
    def __init__(self, num_input_features, dim_kernel, num_embeddings, num_h, dim_W_a, dim_W_c, batch_size):    
        
        super(ActorNet, self).__init__() # this is not really clear to me
        
        self.cnn_embedding = torch.nn.Conv1d(num_input_features, num_embeddings, dim_kernel) # in channel, out channel, kernel size
        print('still to decide the kernel size')
        print('and how do you set the number of filters to D?')
        print('so far I set the number of groups to num_input_features')
        print('doing so, each input featur has its own set of filters (see documentation)')
        self.lstm= torch.nn.LSTM(num_embeddings,  num_h)        
        self.last_item_embedding = torch.nn.Linear(num_input_features+1,  num_embeddings)
        # the plus one up here is the remaining capacity
        print('also, so far you feed to the LSTM both the static and dinamic info.')
        print('and to match the dimension, you pass this info into a fully connected layer')
        print('you will have to change it, so to feed only the static part')
        self.W_a = torch.tensor(torch.rand(dim_W_a,num_embeddings + num_h), requires_grad=True)
        self.v_a = torch.tensor(torch.rand(dim_W_a), requires_grad=True)
        self.v_a = self.v_a[None, :] # I don't know why you have to add a dimension here
        self.W_c = torch.tensor(torch.rand(dim_W_c, 2*num_embeddings), requires_grad=True)
        self.v_c = torch.tensor(torch.rand(dim_W_c), requires_grad=True)
        self.v_c = self.v_c[None, :] # I don't know why you have to add a dimension here
        # last item defines the last item you selected. 
        # it is initialized as an empty item.
        
        # here I set the first internal state of the RNN.
        # I set it here and not in forwards otherwise it start with different h_0
        # For now it's zero-initialied, but maybe there is something smarter 
        # that can be done.
        
        self.batch_size = batch_size 
        self.num_layers = 1
        self.num_directions = 1
        self.num_h = num_h
        
        print('Xavier Initialization still to do')
        #print('first')
        #print(list(self.W_a))
        #torch.nn.init.xavier_uniform(self.W_a)
        #print('after')
        #print(list(self.W_a))
        #stop
        
    def forward(self, x, last_item):

        ReLU = torch.nn.ReLU()
        # compuitng h_t
        last_item = self.last_item_embedding(last_item)
        last_item = ReLU(last_item)
        last_item = last_item[None, :, None] # add fake batch dimension and fake Length dimension
        
        # here it is not clear to me why the batch size has to be one?
        # Maybe because here you pass one info at a time??
        h_0 = torch.zeros( self.num_layers * self.num_directions, 1, self.num_h ) 
        c_0 = torch.zeros( self.num_layers * self.num_directions, 1, self.num_h )
        h_t, cell_t = self.lstm( last_item, (h_0,c_0) )  # if you do not provide the initial cell and state, they are by default zero.

        h_t = torch.squeeze(h_t, dim = 0)    # removing the dimension seq_len since I don't know what it does
        h_t = h_t.t()             # transposing so i have features x samples
        # computing x_bar
        x = x[None,:, :] # I don't know why you have to add a dimension here
        #print(x)
        #print(x.size())
        x_bar = self.cnn_embedding(x)
        print(x_bar)
        print(x_bar.size())
        stop
        # concatenation of two tensors
        Concatenations = []
        for x_bar_i in x_bar:
            Concatenations.append(torch.cat((x_bar_i, h_t), 0))
        concatenation_1 = torch.stack(Concatenations, dim = 0)
        concatenation_1 = torch.squeeze(concatenation_1, dim = 2)
        concatenation_1 = concatenation_1.t()
        # computing u_t
        u_t = torch.mm(self.v_a,torch.tanh(torch.mm(self.W_a, concatenation_1))).t()  
        # computing a_t
        a_t = torch.nn.functional.softmax(u_t, dim = 0)        
        # computing the context vector c_t
        C_t_array = []
        index = 0
        for row in x_bar: # apaprently with split you use less grad operations
            C_t_array.append(torch.mul(row, a_t[index]))
            index+=1        
        c_t = torch.stack(C_t_array,dim=0)
        c_t = torch.sum(c_t, dim=0)        
        # concatenation of two tensors
        Concatenations_2 = []
        for x_bar_i in x_bar:
            Concatenations_2.append(torch.cat((x_bar_i, c_t), 0))
        concatenation_2 = torch.stack(Concatenations_2, dim = 0)
        concatenation_2 = torch.squeeze(concatenation_2, dim = 2).t()        
        # computing u_t_tilde
        u_t_tilde = torch.mm(self.v_c,torch.tanh(torch.mm(self.W_c, concatenation_2)))
        # computing actual output
        x = torch.nn.functional.softmax(u_t_tilde, dim = 1)
        x = torch.squeeze(x, dim = 0)

        return x, x_bar

In [16]:
#create an instance
num_objects = 10
Objects = GenerateANewInstance(num_objects)
for obj in Objects:
    if obj.Prize == 0 and obj.Weight == 0:
        last_item = obj

num_input_features = 2 # number of features of an Object
num_embeddings = 32   # number of features for the embedding
num_h = 32             # number of features for the hidden size of the rnn
dim_W_a = 32           # free dimension of matrix W_a
dim_W_c = 32           # free dimension of matrix W_c
dim_kernel = 2        # kernel dimension
tensors = torch.tensor([[1.1, -1.0],[2.0, 15.0],[1.0, -1.0],[20.0, 12.0],[1.0, -1.0],[2.0, 15.0],[1.0, -1.0],[20.0, 12.0],[1.0, -1.0],[2.0, 15.0],[1.0, -1.0]]).float()

Net_Actor = ActorNet(num_input_features, dim_kernel, num_embeddings, num_h, dim_W_a, dim_W_c, len(tensors))
print('number of parameters: ', sum(p.numel() for p in Net_Actor.parameters() if p.requires_grad))
last_item = torch.tensor([0,0,1]).float()
output = Net_Actor(tensors,last_item)
print('\n\noutput')
print(output)

still to decide the kernel size
and how do you set the number of filters to D?
so far I set the number of groups to num_input_features
doing so, each input featur has its own set of filters (see documentation)
also, so far you feed to the LSTM both the static and dinamic info.
and to match the dimension, you pass this info into a fully connected layer
you will have to change it, so to feed only the static part
Xavier Initialization still to do
('number of parameters: ', 8736)


/Users/BigBamboo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/BigBamboo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/BigBamboo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/Users/BigBamboo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().

RuntimeError: input.size(-1) must be equal to input_size. Expected 32, got 1

Critic network. 
The critic network takes as input the probabilities of the actor network and the embeddings input (x_bar). 
These two are multiplied to obtain a weighted sum of the embedded inputs.
Then, they are fed into a network with 2 layers. 
The first layer is dense with ReLU activation and the second one is a linear layer with single output (which is the expected reward). 

In [46]:
#class Critic(torch.nn.Module):
#    
#    def __init__(self, num_input_features, num_inbetween_features,len_batch):    
#        
#        super(Critic, self).__init__() # this is not really clear to me
#        self.FirstLayer = torch.nn.Linear(num_input_features, num_inbetween_features)
#        self.SecondLayer = torch.nn.Linear(num_inbetween_features*len_batch,  num_inbetween_features*len_batch)        
#        self.SecondLayer_1 = torch.nn.Linear(num_inbetween_features*len_batch,  num_inbetween_features*len_batch)        
#        self.SecondLayer_2 = torch.nn.Linear(num_inbetween_features*len_batch,  num_inbetween_features*len_batch)        
#        self.LastLayer = torch.nn.Linear(num_inbetween_features*len_batch,  1)        
#        torch.nn.init.xavier_normal_(self.FirstLayer.weight)
#        torch.nn.init.xavier_normal_(self.SecondLayer.weight)
#        torch.nn.init.xavier_normal_(self.SecondLayer_1.weight)
#        torch.nn.init.xavier_normal_(self.SecondLayer_2.weight)
#        torch.nn.init.xavier_normal_(self.LastLayer.weight)
#        
#    def forward(self, Outputs, Initial_Features):
#        
#        ReLU = torch.nn.ReLU()
#        x_bar_critic = self.FirstLayer(Initial_Features)
#        Outputs = torch.stack((Outputs), dim=0)
#        OutputMeans = torch.mean(Outputs,0)        
#        Initial_Features_average = []
#        for i in range(len(x_bar_critic)):
#            Initial_Features_average.append(x_bar_critic[i]*OutputMeans[i])
#        Initial_Features_average = torch.cat((Initial_Features_average), dim=0)
#        output = ReLU(Initial_Features_average)
#        output = self.SecondLayer(output)
#        output = ReLU(output)
#        output = self.SecondLayer_1(output)
#        output = ReLU(output)
#        output = self.SecondLayer_2(output)
#        output = ReLU(output)
#        output = self.LastLayer(output)
#        
#        return output

In [47]:
#num_inbetween_features = 32
num_objs = 10
#
#Net_critic = Critic(num_input_features, num_inbetween_features, num_objs+1)
#print('number of critic parameters: ', sum(p.numel() for p in Net_critic.parameters() if p.requires_grad))

In [65]:
Net_Actor = ActorNet(num_input_features, num_embeddings, num_objects+1, len(tensors))
# define for how many different instances it should run, their size and the number of epochs
num_epoch = 250
# create your optimizers
optimizer_actor = torch.optim.Adam(Net_Actor.parameters(), lr=0.01)
Rewards = []
Loss_actor = []
Loss_critic = []
# clipping the gradients
#clip_value = 2
#torch.nn.utils.clip_grad_norm_(Net_Actor.parameters(), clip_value)
# fake instance
Objects = GenerateANewInstance(num_objs)
ObjectsFeatures = torch.tensor([[obj.Prize, obj.Weight] for obj in Objects])

Outputs = []
for e_counter in range(num_epoch):
    optimizer_actor.zero_grad()   # zero the gradient buffers
    total_reward = 0.0
    total_weight = 0.0
    sequence_probability = 1
    Chosen = []
    while True:
        #print('weight', total_weight)
        output = Net_Actor(ObjectsFeatures, total_weight,total_reward)
        Outputs.append(output)
        action_index = np.random.choice(range(num_objs+1),p=output.detach().numpy())
        action_probability = output[action_index]
        sequence_probability = sequence_probability*action_probability
        #print('sequence probability', sequence_probability)
        # perform action
        total_weight+=Objects[action_index].Weight
        total_reward+=Objects[action_index].Prize
        Chosen.append(Objects[action_index])
        last_item = torch.tensor(Objects[action_index].Weight,Objects[action_index].Prize,1-total_weight)
        if total_weight > 1:
            total_reward = -1
            break
        if last_item.Prize == 0 and last_item.Weight == 0:
            total_reward= 5
            break
        total_reward = -1
        break
    #print('sequence probability', sequence_probability)
    #print('Outputs ', Outputs)
    #stop
    target_actor = torch.tensor([5])
    #target_actor.requires_grad = False
    loss_actor = -(torch.tensor(total_reward) - target_actor)*torch.digamma(sequence_probability)
    loss_actor = loss_actor.mean()
    # compute gradient 
    loss_actor.backward()
    # gradient clipping
    clip_value = 1
    for p in Net_Actor.parameters():
        p.register_hook(lambda grad: torch.clamp(grad, -clip_value, clip_value))
    # apply gradient
    optimizer_actor.step()
    # info for plotting
    Rewards.append(total_reward)     
    Loss_actor.append(loss_actor.tolist())
    
print('Last 5 outputs ',Outputs[-5:-1])
R = []
A = []
for i in range(len(Rewards)):
    R.append(Rewards[i])
    A.append(Loss_actor[i])
print('with the fake instance you\'re testing, ')
print('Max reward = 5 ')
plt.figure()
plt.plot(R)
plt.ylabel('Rewards')
plt.figure()
plt.plot(A)
plt.ylabel('Actor loss')

TypeError: __init__() takes exactly 8 arguments (5 given)